In [1]:
'''
读取点云和marks
对点云插值加密并中心化
合并不同文件的marks
'''

import chardet
import numpy as np
import networkx as nx
import scipy.interpolate as spi
import matplotlib.pyplot as plt
from scipy.spatial import KDTree

def detect_encoding(file_path):
    with open(file_path, 'rb') as file:
        raw_data = file.read()
        result = chardet.detect(raw_data)
        return result['encoding']
        
def load_obj_file(file_path, encoding):
    vertices = []
    faces = []
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            for line in file:
                if line.startswith('v '):
                    parts = line.strip().split()
                    vertex = [float(parts[1]), float(parts[2]), float(parts[3])]
                    vertices.append(vertex)
                elif line.startswith('f '):
                    parts = line.strip().split()
                    face = [int(p.split('/')[0]) - 1 for p in parts[1:]]
                    faces.append(face)
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
    except Exception as e:
        print(f"发生错误: {e}")
    return vertices, faces

def load_mark_file(file_path, encoding):
    marks = []
    try:
        with open(file_path, 'r', encoding=encoding) as file:
            for line in file:
                parts = line.strip().split()
                if len(parts) == 3:
                    mark = [float(parts[0]), float(parts[1]), float(parts[2])]
                    marks.append(mark)
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
    except Exception as e:
        print(f"发生错误: {e}")
    return marks

def insert_midpoint_points(vertices, faces, minimum_vertices_number=5000):
    if len(vertices) >= minimum_vertices_number:
        print('原始点云的点数足够多')
        return vertices, faces

    else:
        temp = 0
        while len(vertices) < minimum_vertices_number:
            edge_to_midpoint = {}
            new_points = []
            new_faces = []
            vertex_offset = len(vertices)
    
            for face in faces:
                # Compute midpoints for each edge
                midpoints = []
                for i in range(len(face)):
                    edge = tuple(sorted((face[i], face[(i + 1) % len(face)])))
                    if edge not in edge_to_midpoint:
                        midpoint = np.mean([vertices[edge[0]], vertices[edge[1]]], axis=0)
                        edge_to_midpoint[edge] = vertex_offset + len(new_points)
                        new_points.append(midpoint.tolist())
                    midpoints.append(edge_to_midpoint[edge])
                
                # Original vertices
                v0, v1, v2 = face
                # Midpoints
                m0, m1, m2 = midpoints

                # Create four new faces
                new_faces.append([v0, m0, m2])
                new_faces.append([v1, m1, m0])
                new_faces.append([v2, m2, m1])
                new_faces.append([m0, m1, m2])
    
            vertices.extend(new_points)
            faces = new_faces
            temp += 1

        print(f'使用了 {temp} 轮插值来让点云点数满足要求')
        
    return vertices, faces

def center_vertices(vertices):
    vertices_array = np.array(vertices)
    min_coords = vertices_array.min(axis=0)
    max_coords = vertices_array.max(axis=0)
    center = (min_coords + max_coords) / 2
    centered_vertices = vertices_array - center
    return centered_vertices.tolist()


'''对点云和marks执行姿态调整'''

import numpy as np
from scipy.spatial.transform import Rotation as R
from scipy.interpolate import UnivariateSpline
import os
import matplotlib.pyplot as plt

   
def compute_bounding_box(points):
    """计算点云的包围盒"""
    min_point = np.min(points, axis=0)
    max_point = np.max(points, axis=0)
    bbox_size = max_point - min_point
    bbox_center = (min_point + max_point) / 2.0
    return bbox_size, bbox_center

def extract_top_subcloud(points, bbox_size, bbox_center, height=4.0):
    """提取y坐标在ymax到ymax-3范围内的子点云"""
    min_y = bbox_center[1] + bbox_size[1] / 2.0 - height
    top_subcloud = points[points[:, 1] >= min_y]
    return top_subcloud

def find_best_rotation(points, axis1, axis2, axis3, half_angle_range=int(90), angle_step=1):
    """绕axis1旋转，使得axis2（优先）、axis3方向包围盒尺寸最小，输出旋转矩阵"""
    min_size2 = float('inf')
    min_size3 = float('inf')
    best_rotation_matrix = None

    for angle in np.arange(-half_angle_range, half_angle_range, angle_step):
        # 绕指定轴旋转
        rotation_matrix = R.from_euler(axis1, angle, degrees=True).as_matrix()
        rotated_points = points.dot(rotation_matrix.T)
        
        # 计算包围盒尺寸
        bbox_size, _ = compute_bounding_box(rotated_points)
        
        # 找出目标轴之外的最小尺寸
        if axis2 == 'x':
            size2 = bbox_size[0]  # x方向宽度
        elif axis2 == 'y':
            size2 = bbox_size[1]  # z方向宽度
        elif axis2 == 'z':
            size2 = bbox_size[2]  # z方向宽度

        # 找出目标轴之外的最小尺寸
        if axis3 == 'x':
            size3 = bbox_size[0]  # x方向宽度
        elif axis3 == 'y':
            size3 = bbox_size[1]  # z方向宽度
        elif axis3 == 'z':
            size3 = bbox_size[2]  # z方向宽度

        # 如果当前尺寸比之前的最小尺寸还小，则更新
        if size2 < min_size2:
            min_size2 = size2
            min_size3 = size3
            best_rotation_matrix = rotation_matrix
        elif size2 == min_size2:
            if size3 <min_size3:
                min_size3 = size3
                best_rotation_matrix = rotation_matrix

    return best_rotation_matrix

def flatten_and_analyze_curve(points, z_threshold=0.3, y_offset=1.5):
    """在新x-y平面上分析曲线的曲率，并确定x、z方向。
    
    参数:
    - points: 点云数据，形状为 (n, 3)
    - z_threshold: 用于滤波的z坐标阈值
    - y_offset: 确定用于分裂曲线的横线 y=ymax-y_offset 的偏移量
    """
    # 提取平面内的点
    flat_points = points[np.abs(points[:, 2]) <= z_threshold]

    # 提取 x 和 y 坐标
    x_coords = flat_points[:, 0]
    y_coords = flat_points[:, 1]

    # 对 x_coords 和 y_coords 进行排序以确保 x_coords 严格递增
    sorted_indices = np.argsort(x_coords)
    x_coords = x_coords[sorted_indices]
    y_coords = y_coords[sorted_indices]

    # 确定 ymax - y_offset 这条横线
    ymax = np.max(y_coords)
    y_threshold = ymax - y_offset

    # 找到 y_coords 刚刚超过 y_threshold 的最小和最大索引
    above_threshold_indices = np.where(y_coords > y_threshold)[0]
    if len(above_threshold_indices) == 0:
        raise ValueError("没有点的y坐标超过指定的y_threshold")

    min_index = above_threshold_indices[0]
    max_index = above_threshold_indices[-1]

    # 使用这些索引作为左侧和右侧的分界点
    left_indices = np.where((y_coords <= y_threshold) & (x_coords <= x_coords[min_index]))[0]
    right_indices = np.where((y_coords <= y_threshold) & (x_coords >= x_coords[max_index]))[0]

    left_index = left_indices[np.argmin(np.abs(y_coords[left_indices] - y_threshold))]
    right_index = right_indices[np.argmin(np.abs(y_coords[right_indices] - y_threshold))]

    x2_left, y2_left = x_coords[left_index], y_coords[left_index]
    x2_right, y2_right = x_coords[right_index], y_coords[right_index]

    # 找到曲线的第一个点和最后一个点
    x1, y1 = x_coords[0], y_coords[0]
    x3, y3 = x_coords[-1], y_coords[-1]

    # 计算线段 (x1, y1)-(x2_left, y2_left) 和 (x2_right, y2_right)-(x3, y3) 的斜率
    slope_left = (y2_left - y1) / (x2_left - x1)
    slope_right = (y3 - y2_right) / (x3 - x2_right)

    # 计算两侧线段下方的点数占比
    left_side_points = flat_points[flat_points[:, 0] < x2_left]
    right_side_points = flat_points[flat_points[:, 0] >= x2_right]

    left_under_line = left_side_points[:, 1] < (slope_left * (left_side_points[:, 0] - x1) + y1)
    right_under_line = right_side_points[:, 1] < (slope_right * (right_side_points[:, 0] - x2_right) + y2_right)

    left_under_ratio = np.sum(left_under_line) / len(left_side_points)
    right_under_ratio = np.sum(right_under_line) / len(right_side_points)

    # 确定 x 方向：下凹一侧为 x 正方向
    x_direction = -1 if left_under_ratio > right_under_ratio else 1

    # 复制相同的旋转给 z 方向以保持手性
    z_direction = x_direction

    return x_direction, z_direction

def align_point_cloud(points):
    """处理并旋转点云"""  
    # 计算原始点云的包围盒
    bbox_size, bbox_center = compute_bounding_box(points)

    # 提取顶端子点云1：较短，用于找到最扁取向
    top_subcloud1 = extract_top_subcloud(points, bbox_size, bbox_center, height=3.0)

    # 找到使得点云最扁的旋转角度
    best_rotation_matrix = find_best_rotation(top_subcloud1, 'y', 'x', 'z')

    # 提取顶端子点云2：较长，用于对比唇舌侧的凸性
    top_subcloud2 = extract_top_subcloud(points, bbox_size, bbox_center, height=5.0)
    
    # 将子点云旋转到最佳角度
    rotated_subcloud = top_subcloud2.dot(best_rotation_matrix.T)
    
    # 在新x-y平面上分析曲线并确定x轴方向
    x_direction, z_direction = flatten_and_analyze_curve(rotated_subcloud)

    # 确定最终旋转矩阵
    new_x_axis = best_rotation_matrix[:, 0] * x_direction
    new_y_axis = best_rotation_matrix[:, 1]
    new_z_axis = best_rotation_matrix[:, 2] * z_direction

    best_rotation_matrix_y = np.column_stack((new_x_axis, new_y_axis, new_z_axis))

    # 旋转整个原始点云到新的坐标系
    aligned_points = points.dot(best_rotation_matrix_y.T)

    # 在z轴上进行进一步优化旋转
    best_rotation_matrix_z = find_best_rotation(aligned_points, 'z', 'x', 'y', half_angle_range=int(30))
    aligned_points = aligned_points.dot(best_rotation_matrix_z.T)

    # 在x轴上进行进一步优化旋转
    best_rotation_matrix_x = find_best_rotation(aligned_points, 'x', 'z', 'y', half_angle_range=int(30))
    aligned_points = aligned_points.dot(best_rotation_matrix_x.T)

    # 最终的旋转矩阵
    final_rotation_matrix = best_rotation_matrix_y.T.dot(best_rotation_matrix_z.T).dot(best_rotation_matrix_x.T)

    return aligned_points, final_rotation_matrix


'''
干脆直接从源头将不同marks分割，后续处理方法不变：
第一部分用经典的环形样条插值方法；
检查第二部分中度大于等于3的点，分段做样条插值，再补连剩余边；
把插值线转化为分割条带，实现点云拓扑图的分割。

给分割结果按平均坐标分配标签：
a.y平均值最小的是牙根；
b.剩下类别中y平均值最大是切端；
c.剩下的类别中z最大最小的是近中和远中，x最大和最小的是舌轴面和唇面；
d.剩下的类别中y均值较大的是舌面凹，较小的是肩台。
'''
import networkx as nx
from sklearn.neighbors import KDTree
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np

def generate_topology(marks):
    G = nx.Graph()
    marks_array = np.array(marks)  # 将 marks 转换为 numpy 数组
    tree = KDTree(marks_array)
    
    # 构建拓扑图
    for i, mark in enumerate(marks_array):
        # mark 必须是二维数组 (1, 3)，所以使用 mark.reshape(1, -1)
        distances, indices = tree.query(mark.reshape(1, -1), k=3)  # 找到最近的两个邻居
        for j in indices[0][1:]:  # 跳过自己
            if not G.has_edge(i, j):
                G.add_edge(i, j)
    
    # 获取节点位置 (三维)
    pos = {i: (mark[0], mark[1], mark[2]) for i, mark in enumerate(marks_array)}
    
    return G


import os
import csv
import networkx as nx

def load_G2(file_path):
    # 初始化数据结构
    marks2 = []
    G2 = nx.Graph()
    
    with open(file_path, 'r') as file:
        reader = csv.reader(file)
        next(reader)  # 跳过表头
        for row in reader:
            data_type = row[0]
            if data_type == 'node':
                i, x, y, z = int(row[1]), float(row[3]), float(row[4]), float(row[5])
                marks2.append([x, y, z])
            elif data_type == 'edge':
                source, target = int(row[1]), int(row[2])
                G2.add_edge(source, target)

    return marks2, G2

def find_largest_connected_subgraph(marks2, G2):
    
    # 找到最大的连通子图
    largest_cc_nodes = max(nx.connected_components(G2), key=len)

    # 创建最大连通子图
    largest_subgraph = G2.subgraph(largest_cc_nodes).copy()
    
    # 提取最大连通子图的节点坐标
    max_subgraph_positions = [marks2[node] for node in largest_cc_nodes]
    
    return max_subgraph_positions, largest_subgraph


import numpy as np
from scipy.interpolate import make_interp_spline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx

def interpolate_spline(aligned_marks1, G1):
    """
    对标记点进行样条插值，形成一个闭合曲线。点的顺序由图 G1 确定。
    
    参数:
    aligned_marks1: 一个 Nx3 的数组，包含标记点的坐标。
    G1: NetworkX 图对象，包含标记点的连接顺序。
    
    返回:
    spline_points: 插值后的点的坐标。
    """
    # 确保 aligned_marks1 是一个 numpy 数组
    aligned_marks1 = np.array(aligned_marks1)
    
    # 获取 G1 的边
    edges = list(G1.edges())
    
    # 使用边信息重排标记点
    def get_node_sequence(start_node, graph):
        """
        基于边信息获取标记点的顺序
        """
        sequence = [start_node]
        visited = set([start_node])
        while len(sequence) < len(graph.nodes()):
            current_node = sequence[-1]
            for neighbor in graph.neighbors(current_node):
                if neighbor not in visited:
                    sequence.append(neighbor)
                    visited.add(neighbor)
                    break
        return sequence
    
    # 获取从第一个点开始的顺序
    start_node = list(G1.nodes())[0]
    node_sequence = get_node_sequence(start_node, G1)
    
    # 根据节点顺序重新排列 aligned_marks1
    ordered_marks = np.array([aligned_marks1[i] for i in node_sequence])
    
    # 生成闭环
    closed_marks = np.vstack([ordered_marks, ordered_marks[0]])
    
    # 提取 x, y, z 坐标
    x = closed_marks[:, 0]
    y = closed_marks[:, 1]
    z = closed_marks[:, 2]
    
    # 生成参数 t
    t = np.linspace(0, 1, len(closed_marks))
    
    # 样条插值
    spline_x = make_interp_spline(t, x, k=3)
    spline_y = make_interp_spline(t, y, k=3)
    spline_z = make_interp_spline(t, z, k=3)
    
    # 生成更多的点
    t_new = np.linspace(0, 1, 101)  # 最后一个点会与起点重合，因此实际上是100个点
    spline_x_new = spline_x(t_new)
    spline_y_new = spline_y(t_new)
    spline_z_new = spline_z(t_new)
    
    # 合并第一个点以闭合曲线
    spline_x_new = np.concatenate([spline_x_new, [spline_x_new[0]]])
    spline_y_new = np.concatenate([spline_y_new, [spline_y_new[0]]])
    spline_z_new = np.concatenate([spline_z_new, [spline_z_new[0]]])
    
    spline_points = np.vstack([spline_x_new, spline_y_new, spline_z_new]).T
        
    return spline_points


import numpy as np
from scipy.interpolate import make_interp_spline
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx

def find_intersections_and_paths(aligned_marks2, G2):
    """
    找到 aligned_marks2 中的交点，并确定交点间的路径。如果一条路径中需要通过第三个或更多节点，则删除这条路径。
    
    参数:
    G2: networkx 图，包含图的边信息。
    aligned_marks2: 对应 G2 图中节点的位置。
    
    返回:
    paths: 路径列表。
    intersections: 交点的列表。
    """
    # 识别 G2 中度大于2的点作为交点
    intersections = [node for node, degree in dict(G2.degree()).items() if degree > 2]
    print("节点数：",len(intersections))
    
    # 找到交点间的路径
    paths = []
    
    for i in range(len(intersections)):
        for j in range(i + 1, len(intersections)):
            source = intersections[i]
            target = intersections[j]
            
            try:
                # 计算 source 和 target 之间的最短路径
                path = nx.shortest_path(G2, source=source, target=target)
                
                # 检查路径是否经过其他交点（除了起点和终点）
                # if not any(node in intersections for node in path[1:-1]):
                paths.append(path)  # 仅在路径不经过其他交点时，添加到 paths 列表中
                
            except nx.NetworkXNoPath:
                continue
    print("路径数：",len(paths))
    return paths, intersections

def interpolate_spline_open_path(path, mark_points):
    """
    对给定的路径进行插值。如果路径长度大于3，使用三阶样条插值；否则使用线性插值。
    
    参数:
    path: 要插值的路径，节点的列表。
    mark_points: 对应 G2 图中节点的位置。
    
    返回:
    spline_points: 插值后的点的坐标。
    """
    # 提取路径上的点
    ordered_marks = np.array([mark_points[i] for i in path])
    
    # 提取 x, y, z 坐标
    x = ordered_marks[:, 0]
    y = ordered_marks[:, 1]
    z = ordered_marks[:, 2]
    
    if len(path) > 3:
        # 对于长度大于3的路径，进行三阶样条插值
        t = np.linspace(0, 1, len(path))
        spline_x = make_interp_spline(t, x, k=3)
        spline_y = make_interp_spline(t, y, k=3)
        spline_z = make_interp_spline(t, z, k=3)
        # 插值点数 100
        t_new = np.linspace(0, 1, 100)
        spline_x_new = spline_x(t_new)
        spline_y_new = spline_y(t_new)
        spline_z_new = spline_z(t_new)
        
    else:
        # 对于长度小于等于3的路径，进行线性插值，插值点数 10
        t_new = np.linspace(0, 1, 10)
        spline_x_new = np.interp(t_new, [0, 1], [x[0], x[-1]])
        spline_y_new = np.interp(t_new, [0, 1], [y[0], y[-1]])
        spline_z_new = np.interp(t_new, [0, 1], [z[0], z[-1]])
    
    return np.vstack([spline_x_new, spline_y_new, spline_z_new]).T


import numpy as np
import networkx as nx
from scipy.spatial import cKDTree
from sklearn.neighbors import KDTree

def find_nearby_points(vertices, curve_points, tolerance):
    """
    在曲线附近找到宽度为 tolerance 的点。
    
    参数:
    vertices: 点云数据。
    curve_points: 样条插值生成的曲线点。
    tolerance: 宽度容忍度。
    
    返回:
    nearby_points_indices: 在曲线附近的点的索引。
    """
    # 使用 KDTree 查找邻域
    tree = cKDTree(vertices)
    nearby_points_indices = set()
    
    for point in curve_points:
        indices = tree.query_ball_point(point, tolerance)
        nearby_points_indices.update(indices)
    
    return nearby_points_indices

def generate_graph(vertices, faces):
    """
    生成点云的拓扑图。
    """
    G = nx.Graph()
    for i, vertex in enumerate(vertices):
        G.add_node(i, pos=vertex)
    for face in faces:
        for i in range(len(face)):
            G.add_edge(face[i], face[(i + 1) % len(face)])
    return G

def segment_point_cloud(vertices, faces, spline_points):
    """
    执行点云分割。通过逐步增加tolerance的方式移除spline_points附近的点，直到图被分割为8个大块。
    """
    G = generate_graph(vertices, faces)  # 生成拓扑图

    tolerance = 0.1
    while tolerance <= 1.0:
        nearby_points = find_nearby_points(vertices, spline_points, tolerance)
        G.remove_nodes_from(nearby_points)
        components = list(nx.connected_components(G))
        large_components = [comp for comp in components if len(comp) > 20]  # 规定大于20个点的组件才能被视为真正的组件

        if len(large_components) == 8:
            print(f"分割成功，容差为：{tolerance}")
            
            # y 均值最大和最小的分别是切端和牙根
            sorted_components = sorted(large_components, key=lambda comp: np.mean([vertices[i][1] for i in comp]))
            part1 = sorted_components[0]  # 牙根
            part2 = sorted_components[-1]  # 切端

            # 剩下部分中 z 均值最大和最小的分别是远中和近中
            remaining_parts = sorted_components[1:-1]
            remaining_parts.sort(key=lambda comp: np.mean([vertices[i][2] for i in comp]))
            part3 = remaining_parts[0]  # 近中
            part4 = remaining_parts[-1]  # 远中

            # 剩下部分中 x 均值最大和最小的分别是舌轴面和唇面
            remaining_parts = remaining_parts[1:-1]
            remaining_parts.sort(key=lambda comp: np.mean([vertices[i][0] for i in comp]))
            part5 = remaining_parts[-1]  # 舌轴面
            part6 = remaining_parts[0]  # 唇面

            # 剩下部分中 y 均值较大的是舌面凹，较小的是肩台
            remaining_parts = remaining_parts[1:-1]
            remaining_parts.sort(key=lambda comp: np.mean([vertices[i][1] for i in comp]))
            part7 = remaining_parts[0]  # 肩台
            part8 = remaining_parts[1]  # 舌面窝

            return [part1, part2, part3, part4, part5, part6, part7, part8], nearby_points

        tolerance += 0.02

    print(f"分割失败，组件数：{len(large_components)}")
    return [], nearby_points

def perform_voting(vertices, parts):
    """
    为点云数据中的每个点分配标签，确保每个 part 的标签与其索引一致，同时使用投票机制对未标记的点进行标签分配。
    
    参数:
    vertices: 点云数据，形状为 (n, 3)，每行表示一个点的 (x, y, z) 坐标。
    parts: 列表，包含每个分区的点的索引列表。
    
    返回:
    labels: 点的标签，形状为 (n,)，每个标签是一个整数。
    """
    # 创建 KDTree 以高效地查找邻域
    tree = KDTree(vertices)
    
    # 创建标签数组
    labels = np.zeros(len(vertices), dtype=int)
    
    # 为每个 part 分配标签
    for i, part in enumerate(parts):
        for idx in part:
            labels[idx] = i + 1  # 标签从1开始
    
    # 临时标签数组用于存储投票结果
    temp_labels = np.zeros(len(vertices), dtype=int)
    
    # 设置邻域半径的初始值和最大值
    r_min = 0.2
    r_max = 1
    r_increment = 0.2
    
    # 对每个未标记的点进行投票
    for i, vertex in enumerate(vertices):
        if labels[i] > 0:
            continue  # 跳过已经标记的点
        
        # 初始化邻域半径
        r = r_min
        found_valid_neighbor = False
        
        while r <= r_max:
            # 查找邻居
            neighbors, distances = tree.query_radius([vertex], r=r, return_distance=True)
            neighbors = neighbors[0]  # 提取索引数组
            distances = distances[0]  # 提取距离数组
            
            # 只考虑已标记的邻居进行投票
            valid_neighbors = neighbors[labels[neighbors] > 0]
            valid_distances = distances[labels[neighbors] > 0]
            
            if len(valid_neighbors) > 2:  # 确保有足够的有效邻居
                found_valid_neighbor = True
                break
            r += r_increment
        
        if not found_valid_neighbor:
            continue

        # 如果找到有效邻居，进行标签投票
        neighbor_labels = labels[valid_neighbors]
        unique_labels, counts = np.unique(neighbor_labels, return_counts=True)
        
        if len(unique_labels) > 0:
            # 获取最高计数的标签
            max_count = np.max(counts)
            candidates = unique_labels[counts == max_count]
            
            if len(candidates) > 1:  # 如果有多个标签计数相同，选择平均距离最小的标签
                avg_distances = np.array([np.mean(valid_distances[neighbor_labels == label]) for label in candidates])
                best_label = candidates[np.argmin(avg_distances)]
            else:
                best_label = candidates[0]
            
            temp_labels[i] = best_label
    
    # 更新最终标签
    labels[temp_labels > 0] = temp_labels[temp_labels > 0]
    
    return labels

def save_labeled_point_cloud(vertices, labels, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        for i, vertex in enumerate(vertices):
            if labels[i] > 0:  # 仅保存已标记的点
                file.write(f"{vertex[0]} {vertex[1]} {vertex[2]} {labels[i]}\n")


def process_and_save(obj_file_path, mark_file_path1, mark_file_path2, save_file_path):
    # 检测encoding
    obj_encoding = detect_encoding(obj_file_path)
    mark_encoding1 = detect_encoding(mark_file_path1)

    # 读取点云并插值加密，读取两个marks文件，点云和marks相互配准
    obj_vertices, obj_faces = load_obj_file(obj_file_path, obj_encoding)
    checked_vertices, faces = insert_midpoint_points(obj_vertices, obj_faces)
    centered_vertices = center_vertices(checked_vertices)
    marks1 = load_mark_file(mark_file_path1, mark_encoding1)

    # 姿态调整，生成marks1、2的拓扑图
    aligned_points, final_rotation_matrix = align_point_cloud(np.asarray(centered_vertices))
    aligned_marks1 = np.asarray(marks1).dot(final_rotation_matrix)
    G1 = generate_topology(aligned_marks1)

    # 读取marks2和G2
    aligned_marks2, G2 = load_G2(mark_file_path2)
    _, G2 = find_largest_connected_subgraph(aligned_marks2, G2)

    # marks1插值线上的所有点
    spline_points1 = interpolate_spline(aligned_marks1, G1)

    # 处理marks2
    # 使用 find_intersections_and_paths 获取路径
    paths, intersections = find_intersections_and_paths(aligned_marks2, G2)
    # 对所有路径进行插值
    interpolated_paths = []
    for path in paths:
        interpolated_paths.append(interpolate_spline_open_path(path, aligned_marks2))
    # marks2插值线上的所有点
    spline_points2 = np.unique(np.vstack(interpolated_paths), axis=0)

    # 全部插值线上的点
    spline_points = np.unique(np.vstack([spline_points1, spline_points2]), axis=0)

    # 执行点云分割和标签分配
    parts, points_to_remove = segment_point_cloud(aligned_points, faces, spline_points)
    labels = perform_voting(aligned_points, parts)

    # 保存结果
    save_labeled_point_cloud(aligned_points, labels, save_file_path)


import os

def batch_process_files(obj_file_dir, mark_file_dir1, mark_file_dir2, save_file_dir):
    """
    批量处理数据文件，将结果保存到指定位置。

    参数:
    obj_file_dir: 存放 .obj 文件的目录路径。
    mark_file_dir1: 存放 .mark 文件的第一个目录路径。
    mark_file_dir2: 存放 .mark 文件的第二个目录路径。
    save_file_dir: 保存处理结果的目录路径。
    """
    # 确保保存结果的目录存在
    os.makedirs(save_file_dir, exist_ok=True)

    count1 = 1
    count2 = 0
    
    # 遍历 obj_file_dir 目录中的所有 .obj 文件
    for obj_filename in os.listdir(obj_file_dir):
        if obj_filename.endswith('.obj'):
            print(f"正在处理第{count1}个预备体")
            
            # 构造 obj 文件路径
            obj_file_path = os.path.join(obj_file_dir, obj_filename)
            
            # 构造对应的 mark 文件名
            mark_file_name1 = f"{obj_filename}.mark"
            mark_file_name2 = f"{obj_filename.rstrip(".obj")}.csv"
            
            # 构造 mark 文件路径
            mark_file_path1 = os.path.join(mark_file_dir1, mark_file_name1)
            mark_file_path2 = os.path.join(mark_file_dir2, mark_file_name2)
            
            # 构造保存文件路径
            save_file_name = f"{obj_filename}.txt"
            save_file_path = os.path.join(save_file_dir, save_file_name)
            
            # 检查 mark 文件是否存在
            if os.path.exists(mark_file_path1) and os.path.exists(mark_file_path2):
                # 调用处理函数
                process_and_save(obj_file_path, mark_file_path1, mark_file_path2, save_file_path)
                count2 +=1
            else:
                print(f"Warning: Missing mark file(s) for {obj_filename}")

            print(f"已成功处理{count2}个预备体")
            count1 +=1



In [2]:
'''设置各种地址'''
import os

# 修改当前工作目录，以后输出文件只需要写文件名
new_dir = "D:/李娅宁/肩台外侧点-0715/"
os.chdir(new_dir)
print("修改后的工作目录：", os.getcwd())

# 设置输入文件路径
obj_file_dir = "Aug12/150颗0806新数据"
mark_file_dir1 = "Aug12/150颗0806新数据"
mark_file_dir2 = "Aug15/多分类_0819"
save_file_dir = "Aug22/多分类预处理结果"

# 整
batch_process_files(obj_file_dir, mark_file_dir1, mark_file_dir2, save_file_dir)


修改后的工作目录： D:\李娅宁\肩台外侧点-0715
正在处理第1个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.18
已成功处理1个预备体
正在处理第2个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.18
已成功处理2个预备体
正在处理第3个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.18
已成功处理3个预备体
正在处理第4个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.23999999999999996
已成功处理4个预备体
正在处理第5个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.18
已成功处理5个预备体
正在处理第6个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.19999999999999998
已成功处理6个预备体
正在处理第7个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.34
已成功处理7个预备体
正在处理第8个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.16
已成功处理8个预备体
正在处理第9个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.21999999999999997
已成功处理9个预备体
正在处理第10个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割失败，组件数：5
已成功处理10个预备体
正在处理第11个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.32
已成功处理11个预备体
正在处理第12个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.16
已成功处理12个预备体
正在处理第13个预备体
使用了 1 轮插值来让点云点数满足要求
节点数： 10
路径数： 45
分割成功，容差为：0.2

In [3]:
'''
101不连通
103剩11个节点
106分割失败
109死循环
56死循环
'''
10
17
47
55
68
73
78
102
124
143

143